<a href="https://colab.research.google.com/github/larissavaladao/py6s_harmonize_sample/blob/no_setinel/2.deglint_sampling/deglint_sampling_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [1]:
#import packages used
import ee
import pandas as pd
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import math
import geemap
import os
import math

In [2]:
#authenticate and initialize google earth engine (also necessary for geemap)
ee.Authenticate()
ee.Initialize(project='ee-curuai') #change project name here

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import image collections from GEE

## Landsat 7 BOA

importar coleção com correção Py6S

In [4]:
#Landsat 7
landsat7 = ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD7/ld7_py6s")\
            .select([ 'B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
display(ee.Date(landsat7.first().get('system:time_start')).format().getInfo())
display(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-05-12T13:44:05
2017-09-28T13:50:48


##Landsat 8 BOA

importar coleção com correção Py6S

In [7]:
landsat8 = (ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD8/ld8_py6s")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
display(ee.Date(landsat8.first().get('system:time_start')).format().getInfo())
display(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-05-20T13:50:14
2017-09-27T13:54:12


In [8]:
display('collection size',landsat8.size().getInfo())
display('projection',landsat8.first().select('B4').projection().getInfo())
display('spatial resolution',landsat8.first().select('B4').projection().nominalScale().getInfo())
display('bands',landsat8.first().bandNames().getInfo())

collection size 208
projection {'type': 'Projection', 'crs': 'EPSG:32721', 'transform': [30, 0, 610350, 0, -30, 9780030]}
spatial resolution 30
bands ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# Transform into remote sensing  reflectance and sunglint correction

In [12]:
#dividir imagem corrigida por pi
# Rrs_sat_ac = Rsat_ac / pi

#fazer deglint
#Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
#correção no artigo INPE CURUAI
def deglint (img):
    Rrs = img.divide(math.pi)
    deglint = Rrs.select(['blue','green','red','nir','swir1','swir2'])\
    .subtract(Rrs.select('swir1'))

    return (deglint.copyProperties(img,['system:time_start','CLOUD_COVER',"system:index"]))

## Padronize band names

In [9]:
name_bands = ['blue','green','red','nir','swir1','swir2']

###Landsat 7

In [14]:
#renomear bandas
landsat7_rs = landsat7.map(lambda img: img.select(['B1','B2','B3','B4','B5','B7'])\
                        .rename(name_bands)).map(deglint)

### Landsat 8

In [15]:
landsat8_rs = landsat8.map(lambda img: img.select(['B2','B3','B4','B5','B6','B7'])\
                        .rename(name_bands)).map(deglint)

In [20]:
landsat8_rs.limit(10)

In [21]:
Map = geemap.Map()

imgld7 = landsat7_rs.sort('CLOUD_COVER',True).first()
imgLD8 = landsat8_rs.sort('CLOUD_COVER',True).first()

Map.addLayer(imgld7, {'bands':['red','green','blue'], 'min':0,'max':0.02}, str(imgld7.get('system:index').getInfo()))
Map.addLayer(imgLD8, {'bands':['red','green','blue'], 'min':0,'max':0.02}, str(imgLD8.get('system:index').getInfo()))

Map.centerObject(imgLD8,10)

Map

Map(center=[-2.150324120387019, -55.48666140694086], controls=(WidgetControl(options=['position', 'transparent…

# Merge Harmonized Collections

In [22]:
harmonized_images = landsat8_rs.merge(landsat7_rs).sort('system:time_start')

# Sample multiple points for the entire collection

obtain statistics for the same location of the field point - with a 3 pixel window - and filter out if more than 4 pixels are masked   

In [ ]:
roi_polys = roi_points.map(lambda pt: ee.Feature(pt.geometry().buffer(45).bounds()).set('ID',pt.get('ID'))).filter(ee.Filter.eq('ID',320))
roi_polys.size()

In [ ]:
def zonalStats(ic, fc, params=None):
  _params = {
    'reducer': ee.Reducer.mean(),
    'scale': None,
    'crs': None,
    'bands': None,
    'imgProps': None,
  }
  # // Replace initialized params with provided params.
  # Replace initialized params with provided params
  if params:
     for param, value in params.items():
      _params[param] = value or _params[param]


  # // Map the reduceRegions function over the image collection.
  def reduceRegionfunction (img):
    # // Select bands, set a datetime & timestamp property.
    img = ee.Image(img.select(_params['bands']))\
      .set('datetime', ee.Date(img.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss'))\
      .set('timestamp', img.get('system:time_start'));

    list_props = ee.List(_params['imgProps']).cat(['datetime','timestamp'])
    imgProps = img.toDictionary(list_props)


    # // Subset points that intersect the given image.
    fcSub = fc.filterBounds(img.geometry())

    # // Reduce the image by regions.
    return img.reduceRegions(**{
      'collection': fcSub,
      'reducer': _params['reducer'],
      'scale': _params['scale'],
      'crs': _params['crs']
    }).map(lambda f : f.set(imgProps).set())# // Add metadata to each feature.


  results = ic.map(reduceRegionfunction).flatten().filter(ee.Filter.gt('blue_count',3))

  return results

In [ ]:
params = {
    'reducer': ee.Reducer.mean().combine(ee.Reducer.minMax(),sharedInputs=True)\
    .combine(ee.Reducer.count(),sharedInputs=True)\
    .combine(ee.Reducer.stdDev(),sharedInputs=True)\
    .combine(ee.Reducer.median(),sharedInputs=True).unweighted(),
    'scale': 30,
    'bands': ['blue','green','red','nir'],
    'imgProps': ["SATELLITE_PRODUCT_ID", 'CLOUD_COVER']
  }

In [ ]:
stats = zonalStats(harmonized_images,roi_polys,params)

# Export data as CSV file

In [ ]:
#Exportar tabelas
# Export the image sample feature collection to Drive as a CSV file.
task = ee.batch.Export.table.toDrive(
    collection=stats,
    description='scalled_brdf_bandpassATT_deglint',
    folder='CURUAI_PROCESS',
    fileNamePrefix='scalled_brdf_bandpassATT_deglint',
    fileFormat='CSV',
)
task.start()